In [ ]:
import plotly.express as px
import numpy as np
import h5py 
import seaborn as sns
import pandas as pd
from keras.datasets import mnist
from keras.layers import Input, Dense, Flatten, Conv1D, MaxPooling1D, UpSampling1D, Dense, Reshape, BatchNormalization, GaussianNoise
from keras.models import Model
from keras import backend as K


In [ ]:


def load_raw_read(path):
    f = h5py.File(path, 'r')
    fk = list(f['Raw']['Reads'].keys())
    if len(fk) != 1:
      print("More than one seqence")
    fr = f['Raw']['Reads'][fk[0]]['Signal']
    #ll = list(fr)
    return np.asarray(fr)

def visualize_multiple(rawReads):
    for rR in rawReads:
      fig = px.line(y=rR, labels={"y": "current"})
      fig.update_xaxes(rangeslider_visible=True)
      fig.update_traces(mode='markers+lines')
      fig.show()






In [ ]:
def calc_avg(subs,n,side='front'):
  if side == 'front':
    s = subs[:n]
    sum = 0
    for i in s:
      sum += i
  else:
    s = subs[-n:]
    sum = 0
    for i in s:
      sum += i
  return sum/n

def remove_non_dna(sequence,treshold = 50, stride = 12):
  first_cut = sequence[100:-100]
  avg = calc_avg(first_cut, stride,'front')
  reference_avg = avg
  iterator = stride
  while True:
    new_avg = calc_avg(first_cut[iterator : (iterator + stride)], stride)
    if new_avg - avg > treshold or avg - new_avg > treshold:
      first_cut = first_cut[(iterator + stride) :]
      i = 0
      while reference_avg < first_cut[i] or i < 50:
        i += 1
      first_cut = first_cut[i:]
      break
    iterator += stride
    avg = new_avg
    #print(iterator)

  iterator = stride
  avg = calc_avg(first_cut, stride,'back')
  reference_avg = avg
  while True:
    new_avg = calc_avg(first_cut[-(iterator + stride) : -iterator], stride)
    if new_avg - avg > treshold or avg - new_avg > treshold:
      first_cut = first_cut[:-(iterator + stride)]
      i = 1
      while reference_avg < first_cut[-i] or i < 50:
        i += 1
      first_cut = first_cut[:-i]
      break
    iterator += stride
    avg = new_avg
    #print(iterator)
    
  return first_cut
  

In [ ]:
def normalize(v,med = True):
  df_v = pd.DataFrame(v)
  df_v_or = pd.DataFrame()
  if med:
    df_v_or = df_v.rolling(3,center=True).median()
  else:
    q1 = df_v[0].quantile(0.25)
    q3 = df_v[0].quantile(0.75)
    inter = q3 - q1
    filter = (df_v[0] >= q1 - 1.5 * inter) & (df_v[0] <= q3 + 1.5 *inter)
    df_v_or = df_v.loc[filter]

  df_v_med = df_v_or - 100
  df_v_scale = df_v_med / 800
  return df_v_scale


def prepare_data(v_raw, show = True, norm_type = False):
  v_cut = remove_non_dna(v_raw)
  if (show):
    visualize_multiple([v_cut])
  v_norm = normalize(v_cut,norm_type)
  if (show):
    sns.boxplot(v_norm[0])
  return v_norm


def split_data(v_seq,window_size= 320, stride= 300):
  v_split = []
  seq_len = len(v_seq)
  iterator = 0
  while True:
    if iterator + window_size > seq_len:
      tmp_seq = v_seq[iterator:]
      tmp_seq = np.concatenate((tmp_seq,v_seq[:window_size - (seq_len - iterator)]),axis=0)
      v_split.append(tmp_seq)
      break
    tmp_seq = v_seq[iterator:iterator + window_size]
    v_split.append(tmp_seq)
    iterator += stride
  
  print("Batches created: " + str(len(v_split)))
  return np.array(v_split)

def create_train_test(v_split):
  num_reads = len(v_split)
  num_train = int(0.90 * num_reads)
  v_split_train = v_split[:num_train]
  v_split_test = v_split[num_train:]
  print("Train batches: " + str(len(v_split_train)) + " Test batches: " + str(len(v_split_test)) )
  return v_split_train, v_split_test






**Data loading**

In [ ]:
#Dm1 370 data loading
def load_data(selected_data):
  if selected_data == '370_2014':
    v1_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/370_2014_1/annot/131561ea-d3dd-4e35-bf11-80c0db22f886.fast5')
    v1_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/370_2014_1/annot/536de04b-cd2c-4b96-821e-62fd304c2472.fast5')
    v1_3 = load_raw_read('/content/drive/MyDrive/BakData/dm1/370_2014_1/annot/b14e87e1-eceb-47e2-ba46-9a7e7a10c230.fast5')
    v2_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/370_2014_2/annot/629f8c4a-f1b0-44e2-aafc-4242dae48843.fast5')
    #v2_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/370_2014_2/annot/67c5e2b0-0b84-465a-a1f9-44034683001e.fast5')
    v2_3 = load_raw_read('/content/drive/MyDrive/BakData/dm1/370_2014_2/annot/87edd5c4-2988-4a3d-a879-2dfe8a7f6a8d.fast5')
    return v1_1, v1_2, v1_3, v2_1, v2_3

  if selected_data == '394_2016':
    v1_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/394_2016_1/annot/1ca3725a-6101-4d7e-a16d-5057b61fadee.fast5')
    v1_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/394_2016_1/annot/7477e948-23c5-427f-bbcc-c8256125872d.fast5')
    v1_3 = load_raw_read('/content/drive/MyDrive/BakData/dm1/394_2016_1/annot/dc1a320f-014a-434a-9ce1-9a188bfa5625.fast5')
    v2_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/394_2016_2/annot/2f5d5f2b-2ca5-411e-8be3-399860707bfb.fast5')
    v2_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/394_2016_2/annot/4eb45ebb-6553-448c-9823-7a4040f11f19.fast5')
    v3_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/394_2016_3/annot/b184219e-b21b-4136-a759-34be0e712ebc.fast5')
    v3_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/394_2016_3/annot/e4fa312d-903f-489b-b2ee-f3c9536c3f38.fast5')
    return v1_1, v1_2, v1_3, v2_1, v2_2, v3_1, v3_2

  if selected_data == '84_15_1':
    v1_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_1/annot/19f20510-a1a0-413f-be58-634567681d14.fast5')
    v1_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_1/annot/42d6e949-cb36-43c4-b9de-9541dea91e72.fast5')
    v1_3 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_1/annot/b9f1d5d7-903b-4ee0-875f-9b72d398df48.fast5')
    v1_4 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_1/annot/cb7530ac-8974-4e24-8ba2-2407f446faaf.fast5')
    v1_5 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_1/annot/dd81f005-75aa-4885-bb80-4b72a96a798c.fast5')
    return v1_1, v1_2, v1_3, v1_4, v1_5

  if selected_data == '84_15_2':
    v2_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_2/annot/0b569f5e-a171-468c-be85-5430ae33a508.fast5')
    v2_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_2/annot/58aaf25c-fdb1-487b-9d79-aba658459399.fast5')
    v2_3 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_2/annot/63e573db-f4ee-4f1d-aa70-dc72b1ca01c3.fast5')
    v2_4 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_2/annot/6a21a8fe-2550-498f-95c5-55f63e9ccc30.fast5')
    v2_5 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_2/annot/893e39ee-b26c-44fb-938b-32e059b7d502.fast5')
    v2_6 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_2/annot/d6b53baf-5e69-4070-99f6-b5ff92eb34fd.fast5')
    v2_7 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_2/annot/d925aac9-ea49-497d-9771-e3a8413f7279.fast5')
    return v2_1, v2_2, v2_3, v2_4, v2_5, v2_6, v2_7

  if selected_data == '84_15_3':
    v3_1 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_3/annot/08211ee5-6e4d-4db5-a3f6-01800ef7e881.fast5')
    v3_2 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_3/annot/0e3feab6-2596-4cd5-9830-307ba8985c6f.fast5')
    v3_3 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_3/annot/20914207-c11b-46fc-abf3-f7f4821bb470.fast5')
    v3_4 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_3/annot/452dce8e-4d92-40b7-a42c-92517c767105.fast5')
    v3_5 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_3/annot/63b4bc86-5622-42e2-a9a1-3c180be28001.fast5')
    v3_6 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_3/annot/c4a83453-f871-4cd4-8736-49fb07557005.fast5')
    v3_7 = load_raw_read('/content/drive/MyDrive/BakData/dm1/84_15_3/annot/e919db4b-e9bb-4fc0-b3cc-f16094c3f555.fast5')
    return v3_1, v3_2, v3_3, v3_4, v3_5, v3_6, v3_7

**Data normalization**


In [ ]:
dm1_84_15_1 = load_data('84_15_1')

In [ ]:
dm1_370_14 = load_data('370_2014')

In [ ]:
min = 1000
max = 0
for i in dm1_370_14:
  imax = np.max(i)
  imin = np.min(i)
  if min > imin:
    min = imin
  if max <  imax:
    max =  imax
print([min,max])    

[138, 1092]


In [ ]:
norm_84_15_1_test = []
for i in dm1_84_15_1:
 temp = prepare_data(i,show= False, norm_type= True)
 norm_84_15_1_test.append(temp[0])

In [ ]:
norm_84_15_1 = []
for i in dm1_84_15_1:
 temp = prepare_data(i,show= False)
 norm_84_15_1.append(temp[0])

norm_370_14 = []
for i in dm1_370_14:
 temp = prepare_data(i,show= False)
 norm_370_14.append(temp[0])

In [ ]:
visualize_multiple([norm_370_14[0], norm_370_14[1], norm_370_14[2], norm_370_14[3],  norm_370_14[4]])

In [ ]:
sns.distplot(norm_370_14[0])

In [ ]:
sns.distplot(norm_370_14[3])

In [ ]:
sns.distplot(norm_84_15_1[0])

In [ ]:
sns.distplot(norm_84_15_1[1])

**Spliting into batches**
--------------------------------------------------------------------------------

In [ ]:
split_370_14_train = [] 
split_370_14_test = [] 
for i in norm_370_14:
  train, test = create_train_test(split_data(i))
  split_370_14_train.append(train)
  split_370_14_test.append(test)


Batches created: 100
Train batches: 90 Test batches: 10
Batches created: 203
Train batches: 182 Test batches: 21
Batches created: 190
Train batches: 171 Test batches: 19
Batches created: 332
Train batches: 298 Test batches: 34
Batches created: 331
Train batches: 297 Test batches: 34


In [ ]:
train_data = np.array([ elem for singleList in split_370_14_train for elem in singleList])
valid_data = np.array([ elem for singleList in split_370_14_test for elem in singleList])


In [ ]:
visualize_multiple([norm_370_14[0],norm_370_14[1],norm_370_14[2],norm_370_14[3],norm_370_14[4],norm_370_14[5],])

IndexError: ignored

**CNN Autoencoder**
--------------------------------------------------------------------------------

In [ ]:
input_seq= Input(shape=(320,1))

In [ ]:
# First CNN layer
model_x = Conv1D(filters = 8, kernel_size = 3, activation='relu', padding='same')(input_seq)
print(["First CNN :   ",model_x])
model_x = MaxPooling1D(2, padding='same')(model_x)
print(["First CNN :   " , model_x])


# Second CNN layer
model_x  = Conv1D(filters = 12, kernel_size = 8, activation='relu', padding='same')(model_x)
print(["Second CNN :  " , model_x])
model_x = GaussianNoise(0.1)(model_x)
model_x  = Conv1D(filters = 16, kernel_size = 16, activation='relu', padding='same')(model_x)
print(["Second CNN :  " , model_x])
model_x = BatchNormalization()(model_x)
model_x = MaxPooling1D( 2, padding='same')(model_x)
print(["Second CNN :  " , model_x])

# Third CNN layer

model_x  = Conv1D(filters = 32, kernel_size = 4, activation='relu', padding='same')(model_x)
print(["Third CNN :   " , model_x])
model_x  = Conv1D(filters = 1, kernel_size = 8, activation='relu', padding='same')(model_x)
print(["Third CNN :   " , model_x])
encoded= MaxPooling1D( 2, padding='same')(model_x)
print(["Third CNN :   " , encoded])



#---------------------------------------------#
print('---------------------------------------------')
#---------------------------------------------#


#First DCNN layer
model_x = Conv1D(filters = 1, kernel_size = 8, activation='relu', padding='same')(encoded)
print(["First DCNN :   ",model_x])
model_x  = Conv1D(filters = 32, kernel_size = 4, activation='relu', padding='same')(model_x)
print(["First DCNN :   " , model_x])
model_x = UpSampling1D(2)(model_x)
print(["First DCNN :   " , model_x])

#Second DCNN layer
model_x = Conv1D(filters = 16, kernel_size = 16, activation='relu', padding='same')(model_x)
print(["Second DCNN :  ",model_x])
model_x  = Conv1D(filters = 12, kernel_size = 8, activation='relu', padding='same')(model_x)
print(["Second DCNN :  " , model_x])
model_x = UpSampling1D(2)(model_x)
print(["Second DCNN :  ",model_x])

#Third DCNN layer
model_x = Conv1D(filters = 8, kernel_size = 3, activation='relu', padding='same')(model_x)
print(["Third DCNN :  ",model_x])
model_x = UpSampling1D(2)(model_x)
print(["Third DCNN :  ",model_x])


decoded = Conv1D(1, 1, activation='sigmoid', padding='same')(model_x)
print(["Last CNN:  ",decoded])
autoencoder = Model(input_seq, decoded)
autoencoder.compile(optimizer='nadam', loss='mean_absolute_error')

['First CNN :   ', <KerasTensor: shape=(None, 320, 8) dtype=float32 (created by layer 'conv1d')>]
['First CNN :   ', <KerasTensor: shape=(None, 160, 8) dtype=float32 (created by layer 'max_pooling1d')>]
['Second CNN :  ', <KerasTensor: shape=(None, 160, 12) dtype=float32 (created by layer 'conv1d_1')>]
['Second CNN :  ', <KerasTensor: shape=(None, 160, 16) dtype=float32 (created by layer 'conv1d_2')>]
['Second CNN :  ', <KerasTensor: shape=(None, 80, 16) dtype=float32 (created by layer 'max_pooling1d_1')>]
['Third CNN :   ', <KerasTensor: shape=(None, 80, 32) dtype=float32 (created by layer 'conv1d_3')>]
['Third CNN :   ', <KerasTensor: shape=(None, 80, 1) dtype=float32 (created by layer 'conv1d_4')>]
['Third CNN :   ', <KerasTensor: shape=(None, 40, 1) dtype=float32 (created by layer 'max_pooling1d_2')>]
---------------------------------------------
['First DCNN :   ', <KerasTensor: shape=(None, 40, 1) dtype=float32 (created by layer 'conv1d_5')>]
['First DCNN :   ', <KerasTensor: sha

In [ ]:
autoencoder.fit(train_data,train_data,
epochs=100,
batch_size= 30,
shuffle=False,
validation_data=(valid_data, valid_data),
)

Epoch 1/100
35/35 [==============================] - 36s 40ms/step - loss: 0.0803 - val_loss: 0.0769
Epoch 2/100
35/35 [==============================] - 0s 12ms/step - loss: 0.0710 - val_loss: 0.0776
Epoch 3/100
35/35 [==============================] - 1s 16ms/step - loss: 0.0625 - val_loss: 0.0778
Epoch 4/100
35/35 [==============================] - 1s 15ms/step - loss: 0.0585 - val_loss: 0.0774
Epoch 5/100
35/35 [==============================] - 1s 16ms/step - loss: 0.0550 - val_loss: 0.0778
Epoch 6/100
35/35 [==============================] - 1s 15ms/step - loss: 0.0524 - val_loss: 0.0777
Epoch 7/100
35/35 [==============================] - 1s 16ms/step - loss: 0.0505 - val_loss: 0.0770
Epoch 8/100
35/35 [==============================] - 1s 16ms/step - loss: 0.0493 - val_loss: 0.0746
Epoch 9/100
35/35 [==============================] - 1s 16ms/step - loss: 0.0484 - val_loss: 0.0707
Epoch 10/100
35/35 [==============================] - 0s 14ms/step - loss: 0.0479 - val_loss: 0.067

In [58]:
tmp = autoencoder.predict(valid_data)
vis = np.array([ elem for singleList in tmp[1] for elem in singleList])
visualize_multiple([valid_data[1],vis])